# Code to test HAIM framework on a specific application using HAIM-MIMIC-MM dataset

This notebook is dedicated to demonstrate how an user can use the embedding csv files to make predictions of their task of interest. We here choose to predict a patient's mortality status in the next 48 hours as a demonstration. A patient's death status is defined by their hospital discharge location, if it is morgue, hospice or expired, we consider the patient has died. 

### Project Info
 ->Copyright 2020 (Last Update: June 07, 2022)
 
 -> Authors: 
        Luis R Soenksen (<soenksen@mit.edu>),
        Yu Ma (<midsumer@mit.edu>),
        Cynthia Zeng (<czeng12@mit.edu>),
        Ignacio Fuentes (<ifuentes@mit.edu>),
        Leonard David Jean Boussioux (<leobix@mit.edu>),
        Agni Orfanoudaki (<agniorf@mit.edu>),
        Holly Mika Wiberg (<hwiberg@mit.edu>),
        Michael Lingzhi Li (<mlli@mit.edu>),
        Kimberly M Villalobos Carballo (<kimvc@mit.edu>),
        Liangyuan Na (<lyna@mit.edu>),
        Dimitris J Bertsimas (<dbertsim@mit.edu>),

```
**Licensed under the Apache License, Version 2.0**
You may not use this file except in compliance with the License. You may obtain a copy of the License at
https://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
```

### Requires 
```
 -> Previously generated pickle files from HAIM-MIMIC-MM Dataset
```

## I. Import Packages

In [1]:
# create new environment
!python -m venv haim_env
!source haim_env/bin/activate  # Linux/Mac
# or haim_env\Scripts\activate  # Windows

# Install compatible packages
!pip install numpy==1.23.5
!pip install scikit-learn==1.2.2
!pip install xgboost==1.7.6
!pip install imbalanced-learn==0.12.4


/var/lib/oar/.batch_job_bashrc: line 5: /home/abajrach/.bashrc: No such file or directory
/var/lib/oar/.batch_job_bashrc: line 5: /home/abajrach/.bashrc: No such file or directory
/var/lib/oar/.batch_job_bashrc: line 5: /home/abajrach/.bashrc: No such file or directory
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 17.1 MB 2.6 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.23.5 which is incompatible.
/var/lib/oar/.batch_job_bashrc: line 5: /home/abajrach

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import xgboost as xgb
import numpy as np

## II. Reading Data and Constructing Prediction Target

In [3]:
import pandas as pd

fname = "fusion_emb.csv"

# Load the CSV
df = pd.read_csv(fname)

# Create filtered copies safely
df_death_small48 = df[(df['img_length_of_stay'] < 48) & (df['death_status'] == 1)].copy()
df_alive_big48 = df[(df['img_length_of_stay'] >= 48) & (df['death_status'] == 0)].copy()
df_death_big48 = df[(df['img_length_of_stay'] >= 48) & (df['death_status'] == 1)].copy()

# Assign target labels
df_death_small48['y'] = 1
df_alive_big48['y'] = 0
df_death_big48['y'] = 0

# Concatenate subsets
df = pd.concat([df_death_small48, df_alive_big48, df_death_big48], axis=0)

df = df.fillna(0)

# Drop unwanted columns
df = df.drop(
    ['img_id', 'img_charttime', 'img_deltacharttime', 
     'discharge_location', 'img_length_of_stay', 'death_status',
     'split', 'PerformedProcedureStepDescription', 'ViewPosition', 'Support Devices', 'Fracture'], 
    axis=1
)

# Optional: reset index
df = df.reset_index(drop=True)

print(df.head())

   haim_id  de_0  de_1  de_2  de_3  de_4      vd_0      vd_1      vd_2  \
0       33  81.0     0     0     0     0  0.001103  0.095721  0.297696   
1       47  89.0     0     0     0     0  0.002032  0.026490  0.366769   
2       96  86.0     0     0     0     0  0.002315  0.139908  0.113992   
3       96  86.0     0     0     0     0  0.000000  0.134944  0.195479   
4      110  53.0     0     0     0     0  0.000000  0.044526  0.280396   

       vd_3  ...  Edema  Enlarged Cardiomediastinum  Lung Lesion  \
0  0.003334  ...   -1.0                        -1.0          0.0   
1  0.014916  ...    0.0                         0.0          0.0   
2  0.004673  ...    0.0                         0.0          0.0   
3  0.015708  ...    0.0                         0.0          0.0   
4  0.007477  ...    0.0                         0.0          0.0   

   Lung Opacity  No Finding  Pleural Effusion  Pleural Other  Pneumonia  \
0           1.0         0.0               1.0            0.0        1.0

## III. Training/Testing Set Split

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

seed = 42

# Start from df (already cleaned and with 'y')
x_y = df.copy()
x_y = x_y.astype('float32')

# Drop rows with NaNs first
x_y = x_y[~x_y.isna().any(axis=1)]

# Unique IDs from clean data
pkl_list = x_y['haim_id'].unique().tolist()

# Train-test split of IDs
train_id, test_id = train_test_split(pkl_list, test_size=0.3, random_state=seed)

# Train/test sets
train_set = x_y[x_y['haim_id'].isin(train_id)]
test_set = x_y[x_y['haim_id'].isin(test_id)]

# Features and labels
y_train = train_set['y']
y_test = test_set['y']

x_train = train_set.drop(['y','haim_id'], axis=1)
x_test = test_set.drop(['y','haim_id'], axis=1)

print('train, test shapes', x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print('train set, death outcome case = %s, percentage = %.2f' % (y_train.sum(),  y_train.sum()/len(y_train)))
print('test set, death outcome case = %s, percentage = %.2f' % (y_test.sum(),  y_test.sum()/len(y_test)))


train, test shapes (718, 2552) (434, 2552) (718,) (434,)
train set, death outcome case = 16.0, percentage = 0.02
test set, death outcome case = 5.0, percentage = 0.01


In [5]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 1. Scale features (important for PCA)
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# 2. Apply PCA
n_components = 100  # choose number of components or explain variance threshold
pca = PCA(n_components=n_components, random_state=42)
x_train = pca.fit_transform(x_train_scaled)
x_test = pca.transform(x_test_scaled)

# Optional: check explained variance
explained_var = pca.explained_variance_ratio_.sum()
print(f"Total explained variance with {n_components} components: {explained_var:.2f}")
print("Shape after PCA:", x_train.shape, x_test.shape)


Total explained variance with 100 components: 0.90
Shape after PCA: (718, 100) (434, 100)


In [6]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
x_train, y_train = sm.fit_resample(x_train, y_train)

## IV: Model Training

In [7]:
cv_folds = 5
gs_metric = 'roc_auc'
param_grid = {'max_depth': [5, 6, 7, 8],
             'n_estimators': [200, 300],
             'learning_rate': [0.3, 0.1, 0.05],
             }

est = xgb.XGBClassifier(verbosity=0, scale_pos_weight = (len(y_train) - sum(y_train))/sum(y_train), seed = 42,
                        eval_metric='logloss')

gs = GridSearchCV(estimator = est, param_grid=param_grid, scoring=gs_metric, cv= cv_folds)
gs.fit(x_train, y_train)

y_pred_prob_train = gs.predict_proba(x_train)
y_pred_train = gs.predict(x_train)

y_pred_prob_test = gs.predict_proba(x_test)
y_pred_test = gs.predict(x_test)

## V:Reporting Performance Metrics

In [8]:
from sklearn import metrics

f1_train = metrics.f1_score(y_train, y_pred_train, average='macro')
accu_train = metrics.accuracy_score(y_train, y_pred_train)
accu_bl_train = metrics.balanced_accuracy_score(y_train, y_pred_train)
auc_train = metrics.roc_auc_score(y_train, y_pred_prob_train[:, 1])
conf_matrix_train = metrics.confusion_matrix(y_train, y_pred_train)

In [9]:
print(f'F1 Score for Training Set is: {f1_train}')
print(f'Accuracy for Training Set is: {accu_train}')
print(f'Balanced Accuracy for Training Set is: {accu_bl_train}')
print(f'AUC for Training Set is: {auc_train}')
print(f'Confusion Matrix for Training Set is: {conf_matrix_train}')

F1 Score for Training Set is: 1.0
Accuracy for Training Set is: 1.0
Balanced Accuracy for Training Set is: 1.0
AUC for Training Set is: 1.0
Confusion Matrix for Training Set is: [[702   0]
 [  0 702]]


In [10]:
f1_test = metrics.f1_score(y_test, y_pred_test, average='macro')
accu_test = metrics.accuracy_score(y_test, y_pred_test)
accu_bl_test = metrics.balanced_accuracy_score(y_test, y_pred_test)
auc_test = metrics.roc_auc_score(y_test, y_pred_prob_test[:, 1])
conf_matrix_test = metrics.confusion_matrix(y_test, y_pred_test)

In [11]:
print(f'F1 Score for Testing Set is: {f1_test}')
print(f'Accuracy for Testing Set is: {accu_test}')
print(f'Balanced Accuracy for Testing Set is: {accu_bl_test}')
print(f'AUC for Testing Set is: {auc_test}')
print(f'Confusion Matrix for Testing Set is: {conf_matrix_test}')

F1 Score for Testing Set is: 0.49710312862108924
Accuracy for Testing Set is: 0.988479262672811
Balanced Accuracy for Testing Set is: 0.5
AUC for Testing Set is: 0.7263403263403263
Confusion Matrix for Testing Set is: [[429   0]
 [  5   0]]
